# Tests

## Imports and stuff

In [1]:
import sys

def append_path(s):
    if s in sys.path:
        return
    sys.path.append(s)

append_path("..")
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
from numpy.random import default_rng
rng = default_rng()
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import networkx as nx
import torch
from torch import Tensor


In [3]:
import ml_lib


## Datasets

In [4]:
from ml_lib.datasets import register as dataset_register, transform_register

In [5]:
dataset0 = dataset_register["Torus4D"](1000)

In [6]:
dataset1 = transform_register["CacheTransform"](dataset0)

In [7]:
dataset1[0]

tensor([[-1.0065,  0.0609,  0.2676, -0.9727]])

In [8]:
dataset2 = transform_register["RenameTransform"](dataset1, {"_": "x"})
dataset2[0]

datapoint(x=tensor([[-1.0065,  0.0609,  0.2676, -0.9727]]))

In [9]:
dataset3 = transform_register["RenameTransform"](dataset2, {"x": "x", 0:"y"})
dataset3[0]

datapoint(x=tensor([[-1.0065,  0.0609,  0.2676, -0.9727]]), y=tensor([[-1.0065,  0.0609,  0.2676, -0.9727]]))

In [10]:
normalize = lambda x : x / x.norm()
dataset4 = transform_register["MultipleFunctionTransform"](dataset3, {"x": (normalize, "x"), "gt": (lambda x:x, "y")})
dataset4[0]

datapoint(x=tensor([[-0.7056,  0.0427,  0.1876, -0.6820]]), gt=tensor([[-1.0065,  0.0609,  0.2676, -0.9727]]))

great!

## Environments

In [11]:
from ml_lib.environment import Environment, Scope, scopevar_of_str, str_of_scopevar, HierarchicEnvironment, ScopedEnvironment

In [12]:
scopevar_of_str("a/b/c")

(('a', 'b'), 'c')

In [13]:
str_of_scopevar(('a', 'b'), 'c')

'a/b/c'

In [14]:
env = Environment()

In [15]:
env

Environment(defaultdict(<class 'dict'>, {'_ipython_canary_method_should_not_exist_': {}, '_ipython_display_': {}, '_repr_mimebundle_': {}}))

In [16]:
env.record("hello", 1)

In [17]:
env.data

defaultdict(dict,
            {'_ipython_canary_method_should_not_exist_': {},
             '_ipython_display_': {},
             '_repr_mimebundle_': {},
             '_repr_html_': {},
             '_repr_markdown_': {},
             '_repr_svg_': {},
             '_repr_png_': {},
             '_repr_pdf_': {},
             '_repr_jpeg_': {},
             '_repr_latex_': {},
             '_repr_json_': {},
             '_repr_javascript_': {},
             'hello': {(): 1}})

In [18]:
env.record("world", 4, ("some", "scope"))
env.data

defaultdict(dict,
            {'_ipython_canary_method_should_not_exist_': {},
             '_ipython_display_': {},
             '_repr_mimebundle_': {},
             '_repr_html_': {},
             '_repr_markdown_': {},
             '_repr_svg_': {},
             '_repr_png_': {},
             '_repr_pdf_': {},
             '_repr_jpeg_': {},
             '_repr_latex_': {},
             '_repr_json_': {},
             '_repr_javascript_': {},
             'hello': {(): 1},
             'world': {('some', 'scope'): 4}})

In [19]:
env.get("hello")

1

In [20]:
env.get("world")

4

In [21]:
#import pdb; pdb.set_trace()
env.get("world", scope=("some",))

4

In [22]:
env.get("world", scope=("soe",))

In [23]:
hier_env = HierarchicEnvironment(parent=env)

In [24]:
hier_env.record("world", 2, ("some",))
hier_env.get("world", scope=("some",))


2

In [25]:
hier_env.get("world", scope=("some","scope"))


4

In [26]:
def f(world, hello=6, magic=3):
    print(world, hello, magic)
    return 1

In [27]:
hier_env.run_function(f)

2 1 3


1

## Model

In [28]:
from ml_lib.models import Model, Supervised, Hyperparameter
from ml_lib.models.layers import MLP
class SimpleMLPModel(Supervised):
    
    dimensions: Hyperparameter[list[int]]

    inner: MLP

    def __setup__(self):
        self.inner = MLP(*self.dimensions, batchnorm=False)

    def forward(self, x):
        return self.inner(x)

    def loss_fun(self, x, gt):
        return (x - gt).square().sum()

model = SimpleMLPModel(dimensions=[4, 5, 10, 4, 4])
print(model)
print(torch.nn.Module.__repr__(model))
model.model_name = "test_model"

SimpleMLPModel(
    dimensions=[4, 5, 10, 4, 4],
)
SimpleMLPModel(
  (inner): MLP(
    (linear_0): Linear(in_features=4, out_features=5, bias=True)
    (activation_0): ReLU()
    (linear_1): Linear(in_features=5, out_features=10, bias=True)
    (activation_1): ReLU()
    (linear_2): Linear(in_features=10, out_features=4, bias=True)
    (activation_2): ReLU()
    (linear_3): Linear(in_features=4, out_features=4, bias=True)
  )
)


## Saving and loading to database

In [29]:
from sqlalchemy import create_engine
from sqlalchemy.orm import Session
from ml_lib.experiment_tracking import create_tables, Model as Database_Model


In [30]:
model.get_model_type()

'SimpleMLPModel'

In [31]:
model.to_database_object()

Model(id= None, model_type= SimpleMLPModel, name= test_model, description= None)

In [32]:
print_requests = False
db_engine = create_engine("sqlite:////tmp/test.db", echo=print_requests)
create_tables(db_engine)
with Session(db_engine) as db_session:
    model.save_to_database(db_session, replace=True)
    #db_session.commit()
    print(model.get_database_object(db_session))
    model_object  = db_session.get(Database_Model, model.id)
    db_session.commit()
model.id

Model(id= 1, model_type= SimpleMLPModel, name= test_model, description= None)


1

In [38]:
with Session(db_engine) as db_session:
    model_object  = db_session.get(Database_Model, model.id)
    print(model_object)
    print(type(db_session))


Model(id= 1, model_type= SimpleMLPModel, name= test_model, description= None)
<class 'sqlalchemy.orm.session.Session'>


## Training

In [34]:
from ml_lib.models import Model
from ml_lib.pipeline import Trainer
from ml_lib.pipeline.training_hooks import TqdmHook, LoggerHook, CurveHook
from torch.utils.data import DataLoader

In [35]:
loader = DataLoader(dataset4, batch_size=None, shuffle=True)

In [36]:
next(iter(loader))

datapoint(x=tensor([[-0.4805, -0.5292, -0.5577,  0.4219]]), gt=tensor([[-0.6796, -0.7485, -0.7889,  0.5968]]))

In [37]:
import logging; logging.basicConfig(level=logging.INFO, force=True)

curve = CurveHook()

trainer = Trainer(model, loader, n_epochs = 4, device="cpu",
    step_hooks = [ 
        TqdmHook(),
        LoggerHook(interval=10),
        curve
    ]
                                                                      
                 )
trainer.train()

  0%|          | 0/4000 [00:00<?, ?it/s]

INFO:root:loss= 0.9821060299873352, 
INFO:root:loss= 1.5564428567886353, 
INFO:root:loss= 4.05483341217041, 
INFO:root:loss= 4.092958450317383, 
INFO:root:loss= 2.9928622245788574, 
INFO:root:loss= 3.238398313522339, 
INFO:root:loss= 3.6726975440979004, 
INFO:root:loss= 2.4147825241088867, 
INFO:root:loss= 2.198533058166504, 
INFO:root:loss= 3.6477832794189453, 
INFO:root:loss= 2.195890188217163, 
INFO:root:loss= 2.3298072814941406, 
INFO:root:loss= 1.7302006483078003, 
INFO:root:loss= 2.6889190673828125, 
INFO:root:loss= 1.8359925746917725, 
INFO:root:loss= 2.875742197036743, 
INFO:root:loss= 2.592878818511963, 
INFO:root:loss= 2.433959484100342, 
INFO:root:loss= 2.3393778800964355, 
INFO:root:loss= 2.2343454360961914, 
INFO:root:loss= 1.4649765491485596, 
INFO:root:loss= 1.2942402362823486, 
INFO:root:loss= 1.3919920921325684, 
INFO:root:loss= 1.1436883211135864, 
INFO:root:loss= 2.212385892868042, 
INFO:root:loss= 2.16654109954834, 
INFO:root:loss= 1.1187154054641724, 
INFO:root:los

In [ ]:
curve.draw()

In [ ]:
pip install matplotlib